**Çalışmayı beğenirseniz oylamayı unutmayın^_____^!**



**Önemli Not:** Farklı Temel Bileşen sayılarına göre elde ettiğim başarı oranlarını buraya not etmek istiyorum. Temel Bileşen sayılarına nasıl karar derdiğimi [Uygun Sayıda Temel Bileşenin Bulunması](#4e2.) bölümünde anlatmıştım. Bileşen sayısı artırmak başarı oranına katkıda bulunmuyor; aksine gürültüye sebep olduğu için başarı oranını düşürüyor olabilir. Yarışma veri setinde, farklı Temel Bileşen sayısına göre elde ettiğim sonuçlar şöyledir:

* Temel Bileşen sayısı: 150, yarışma veri seti üzerinde başarı oranı:0.96714
* Temel Bileşen sayısı: yine 150, yarışma veri seti üzerinde başarı oranı:0.96928
* Temel Bileşen sayısı: 200, yarışma veri seti üzerinde başarı oranı:0.95157
* Temel Bileşen sayısı: 40, yarışma veri seti üzerinde başarı oranı:0.97700
* Temel Bileşen sayısı: 35, yarışma veri seti üzerinde başarı oranı: 0.97471
* PCA projeksiyonunu bütün veri setinde eğitip 35 Temel Bileşen kullanınca başarı oranı:0.98014

# <a class="anchor" id="0."></a>**İçindekiler**

1. [Çalışmanın Özeti](#1.)
1. [Veri Setinin Yüklenmesi](#2.)
1. [Yardımcı Fonksiyon: show_digit_matrix()](#3.)
1. [Veri Setinde Yer Alan El Yazısı Rakamlarından Bazılarının Gösterimi](#4.)
1. [Temel Bileşen Analizi-Principle Component Analiysis(PCA)](#4e1.)
1. * [Uygun Sayıda Temel Bileşenin Bulunması](#4e2.)
1. [Eğitim Veri Setinde PCA+SVC Kullanılması](#5.)
1. [Yarışma Veri Setinde PCA+SVC Kullanılması](#6.)
1. [Yarışma Gönderisinin Kaydedilmesi](#7.)
1. [Pipelines İle İş Akışının Otomatikleştirilmesi](#8.)
1. [Pipeline İle PCA ve LogisticRegression](#9.)
* * [Pipeline İle Yarışma Verisi Kaydediliyor](#10.)


[İçindekiler Menüsüne Git](#0.)
 
# <a class="anchor" id="1."></a> **1.Çalışmanın Özeti**

Bu çalışmada bilgisayarlı görme problemleri için giriş seviyesinde bir veri seti olan MNIST el yazması veri seti üzerinde PCA ve SVC ile sınıflandırma işlemi yapılmıştır. Ayrıcı, Kaggle yarışması için ayrılan veri seti üzerinde yapılan sınıflandırma sonuçları yarışma gönderisi olarak kaydedilmiştir.. 

In [ ]:
#Gerekli Kütüphaneler Yükleniyor

import numpy as np # linear algebra
import pandas as pd # Veri işleme

import matplotlib.pyplot as plt

#Görüntü işleme
import cv2

#Makine Öğrenmesi
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.feature_selection import SelectKBest

from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

#Sistem Kütüphaneleri
import os
import warnings
print(os.listdir("../input"))
#Uyarıları kapatılıyor
warnings.filterwarnings('ignore')


[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="2."></a> **2. Veri Setinin Yüklenmesi** 

In [ ]:
#Eğitim veri seti yükleniyor
train=pd.read_csv("../input/train.csv",dtype="uint8")
train.head()

In [ ]:
#Çıktı sütün vektörü elde ediliyor
y=train['label']

#Veri matrisi elde ediliyor
data=train.drop('label',axis=1).values

#Veri matrisinden sadece değerler alınıyor
X=data[:,0:]

In [ ]:
print("çıktı y.shape:",y.shape)
print("veri X.shape :",X.shape)

In [ ]:
#Eğitim için veri setinin hepsini almak yerine istediğimiz kısmını alıyoruz
#Son olarak veri setinin hepsini alıyoruz.
#Son durum itibariyle veri setinin hepsi alınmıştır.
#Kendiniz çalışırken n_samples değerini değitirip sonuçları gözlemleyebilirsiniz.

n_samples=y.shape[0]#Veri setinin hepsi alınıyor
y=y[:n_samples]
X=X[:n_samples]
print("çıktı y.shape:{}".format(y.shape))
print("veri X.shape:{}".format(X.shape))

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="3."></a>**3. Yardımcı Fonksiyon: show_digit_matrix()** 

In [ ]:
def show_digit_matrix(digit, n=10):
    v_images=[]
    n=n
    count=0
    for i in range(0,n):
        h_images=list()
        for j in range(0,n):
            h_images.append(digit[count].reshape(28,28))
            count+=1
        h=np.hstack((h_images))
        v_images.append(h)
    image_matrix=np.vstack((v_images))
    
    fig, axarr = plt.subplots(1, 1, figsize=(12, 12))
    plt.imshow(image_matrix,cmap='gray')


<[Önceki](#3.) | [İçindekiler Menüsüne Git](#0.) | [Sonraki](#4e1.)>

# <a class="anchor" id="4."></a>**4. Veri Setinde Yer Alan El Yazısı Rakamlarından Bazılarının Gösterimi** 

In [ ]:
show_digit_matrix(digit=X, n=20)

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="4e1."></a>**5. Temel Bileşen Analizi-Principle Component Analiysis(PCA)**


In [ ]:
pca=PCA(n_components=2, whiten=True)
pca.fit(X)
X_pca=pca.transform(X)

In [ ]:

plt.figure(1, figsize=(12,8))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y, s=10,cmap=plt.get_cmap('jet',10))
plt.colorbar()

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="4e2."></a>**5.1. Uygun Sayıda Temel Bileşenin Bulunması** 

In [ ]:
pca=PCA()
pca.fit(X)

In [ ]:
plt.figure(1,figsize=(12,8))
plt.xticks(np.arange(0, 800, 30.0))
plt.plot(pca.explained_variance_,linewidth=2)


Yukarıdaki grafikte 30 Temel Bileşen kadar, veriyi çok yüksek düzeyde temelsil etme yeteneğine sahip olduğu görülüyor. 150 ve daha fazla bileşenin, veriyi temsil etme yeteneklerinin aynı olduğu görülüyor. 

[İçindekiler Menüsüne Git](#0.)
 
<a class="anchor" id="5."></a>**6. Eğitim Veri Setinde PCA+SVC Kullanılması**

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, #Veri matrisi
                                                  y, #Çıktı vektörü
                                                  stratify=y, #Her sınıftan eşit oranda ayrıştırma için
                                                  test_size=0.3, #%30 test ve %70 eğitim için ayrıştır
                                                  random_state=42 #rasgele sayı çekirdeği
                                                 )
print("Veri seti eğitim ve test olarak ayrıştırıldı")

In [ ]:
#Kullanılacak temel bileşen sayısı
n_components=35

In [ ]:
pca=PCA(n_components=n_components, whiten=True)
pca.fit(X_train)
X_train_pca=pd.DataFrame(pca.transform(X_train))
X_test_pca=pd.DataFrame(pca.transform(X_test))
print("Eğitim veri seti için PCA dönüşümü gerçekleştirildi")

In [ ]:
params=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1.0, 10.0], 'kernel': ['rbf']}
       ]
#Çok uzun sürdüğü için GridSearchCV kısmını pasif hale getirdim.
#Siz isterseniz çalışmayı çatallayıp(Fork) deneyebilirsiniz. 
#En iyi sonuç veren parametreler: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
#clf=GridSearchCV(SVC(), params, cv=5)

clf=SVC(C=10, gamma=0.01, kernel="rbf")
clf.fit(X_train_pca, y_train)
y_pred1=clf.predict(X_test_pca)
print("Sınıflandırıcı:{}".format(clf.__class__.__name__))

print("Başarı oranı:{}".format(accuracy_score(y_pred1, y_test)))
print("Karışıklık Matrisi:\n{}".format(confusion_matrix(y_pred1, y_test)))
print("Sınıflandırma Raporu:\n{}".format(classification_report(y_pred1, y_test)))

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="6."></a>**7. Yarışma Veri Setinde PCA+SVC  Kullanılması**

In [ ]:
competion_dataset=pd.read_csv("../input/test.csv",dtype="uint8")
competion_dataset=competion_dataset.values
competion_dataset=competion_dataset[:,0:]
print("Yarışma competion_dataset.shape :",competion_dataset.shape)

In [ ]:
pca=PCA(n_components=n_components, whiten=True)
pca.fit(X)
print("PCA eğitimi için 'eğitim'  veri setinin hepsi kullanıldı")

In [ ]:
X_pca=pca.transform(X)
print("Eğitim veri setinin tümünün PCA dönüşümü gerçekleştirildi")

In [ ]:
competion_dataset_pca=pca.transform(competion_dataset)
print("Yarışma veri seti için PCA dönüşümü gerçekleştirildi")

In [ ]:
clf.fit(X_pca,y)
print("Tüm eğitim veri seti kullanılarak sınıflandırıcı eğitildi")

In [ ]:
print("Yarışma gönderisi hazırlanıyor...")
y_pred2=clf.predict(competion_dataset_pca)
print("Yarışma gönderisi hazır.")

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="7."></a>**8. Yarışma Gönderisinin Kaydedilmesi**  

In [ ]:
file_name="x_pca_{}_svc_mnist.csv".format(n_components)
print(file_name)

In [ ]:
results = pd.Series(y_pred2,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv(file_name,index=False)
print("Yarışma gönderisi kaydedildi.")

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="8."></a>**9. Pipeline İle İş Akışının Otomatikleştirilmesi**  

Makine öğrenmesinin veri setleri üzerinde uygulamasının standart iş akışı vardır. Bu iş akışını otomatikleştirmek için sklearn Pipeline(boru hattı) nesnesini sunmaktadır. Pipeline, makine öğremesi için yapılması gerekli olan ölçeklendirme, özellik çıkarma ve modelleme gibi işlemleri sırasıyla yapmaya olanak sağlar. Pipeline, tüm veri setinde aynı işlemin yapılmasını garanti ederek eğitim ve test verilerinin tutarlı olmasını garanti eder.

In [ ]:
work_flows=list()
work_flows.append(('pca', PCA(n_components=n_components, whiten=True)))
work_flows.append(('svm',SVC(C=10, gamma=0.01, kernel="rbf")))
clf=Pipeline(work_flows)
print("İş akışı oluşturuldu")

In [ ]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("İş akışı eğitildi")

In [ ]:
print("Sınıflandırıcı:{}".format(clf.__class__.__name__))

print("Başarı oranı:{}".format(accuracy_score(y_test, y_pred)))
print("Karışıklık Matrisi:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Sınıflandırma Raporu:\n{}".format(classification_report(y_test, y_pred)))

[İçindekiler Menüsüne Git](#0.)
# <a class="anchor" id="9."></a>**10. Pipeline İle PCA ve LogisticRegression**  

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
features = []
features.append(('pca', PCA(n_components=n_components)))
features.append(('select_best', SelectKBest(k=300)))
feature_union = FeatureUnion(features)
#FeatureUnion iyi sonuç vermedi; daha sonra tekrar denenecek 
# create pipeline
work_flows = []
work_flows.append(('pca', PCA(n_components=n_components, whiten=True)))
work_flows.append(('LogReg', LogisticRegression()))
clf = Pipeline(work_flows)
print("İş akışı oluşturuldu")

In [ ]:
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print("İş akışı eğitildi")

In [ ]:
print("Sınıflandırıcı:{}".format(clf.__class__.__name__))

print("Başarı oranı:{}".format(accuracy_score(y_test, y_pred)))
print("Karışıklık Matrisi:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Sınıflandırma Raporu:\n{}".format(classification_report(y_test, y_pred)))

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="10."></a>**11. Pipeline İle Yarışma Verisi Kaydediliyor**  

In [ ]:
clf.fit(X, y)
print("model eğitildi")

In [ ]:
y_pred2=clf.predict(competion_dataset)
print("Yarışma veri setinde tahmin gerçekleştirildi.")

In [ ]:
file_name="pipeline_pca_{}_lr_mnist.csv".format(n_components)
print(file_name)

In [ ]:
results = pd.Series(y_pred2,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv(file_name,index=False)
print("İkinci yarışma gönderisi kaydedildi.")

Çalışma hakkındaki yorumlarınızı bekliyorum.  Beğendiyseniz oy vermeyi unutmayın ::))